In [3]:
import os, time
import numpy as np
import pandas as pd
from ieeg.auth import Session

from get_iEEG_data import *
from iEEG_helper_functions import *

In [4]:
SPIKES_OUTPUT_DIR = "../../Data/spikes/devin_spikes/"
SYNCHRONY_broadband_DIRECTORY = "../../Data/synchrony/all/broadband"

In [5]:
# Load HUP_implant_dates.xlsx
nina_patients_df = pd.read_excel("../../Data/HUP_implant_dates.xlsx")
# Make the hup_id column integers
nina_patients_df["hup_id"] = nina_patients_df["hup_id"].astype(int)
nina_patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


## Select a batch

In [6]:
batch = nina_patients_df[nina_patients_df["hup_id"] == 143].reset_index(drop=True)
batch

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,143,HUP143_phaseII,2017-07-17,07:15:00,2017-07-29,61.5


In [7]:
print("Using Carlos session")
with open("agu_ieeglogin.bin", "r") as f:
    session = Session("aguilac", f.read())

Using Carlos session


In [12]:
# Iterate through every row in batch
for index, row in batch.iterrows():
    hup_id = row["hup_id"]
    dataset_name = row["IEEG_Portal_Number"]
    print(dataset_name)

    dataset = session.open_dataset(dataset_name)

    all_channel_labels = np.array(dataset.get_channel_labels())
    channel_labels_to_download = all_channel_labels[
        electrode_selection(all_channel_labels)
    ]

    duration_usec = dataset.get_time_series_details(
        channel_labels_to_download[0]
    ).duration
    duration_hours = int(duration_usec / 1000000 / 60 / 60)
    enlarged_duration_hours = duration_hours + 24

    print(f"Opening {dataset_name} with duration {duration_hours} hours")

    # Calculate the total number of 2-minute intervals in the enlarged duration
    total_intervals = enlarged_duration_hours * 30  # 60min/hour / 2min = 30

    synchrony_broadband_vector_to_save = np.full(total_intervals, np.nan)

    # Loop through each 2-minute interval
    for interval in range(717, 727):
        print(f"Getting iEEG data for interval {interval} out of {total_intervals}")
        duration_usec = 1.2e8  # 2 minutes
        start_time_usec = interval * 2 * 60 * 1e6  # 2 minutes in microseconds
        stop_time_usec = start_time_usec + duration_usec

        try:
            ieeg_data, fs = get_iEEG_data(
                "aguilac",
                "agu_ieeglogin.bin",
                dataset_name,
                start_time_usec,
                stop_time_usec,
                channel_labels_to_download,
            )
            fs = int(fs)
        except Exception as e:
            # handle the exception
            print(f"Error: {e}")
            break

        # Drop rows that has any nan
        ieeg_data = ieeg_data.dropna(axis=0, how="any")
        if ieeg_data.empty:
            print("Empty dataframe after dropping nan, skip...")
            continue

        good_channels_res = detect_bad_channels_optimized(ieeg_data.to_numpy(), fs)
        good_channel_indicies = good_channels_res[0]
        good_channel_labels = channel_labels_to_download[good_channel_indicies]
        ieeg_data = ieeg_data[good_channel_labels].to_numpy()

        # Check if ieeg_data is empty after dropping bad channels
        if ieeg_data.size == 0:
            print("Empty dataframe after dropping bad channels, skip...")
            continue

        ieeg_data = common_average_montage(ieeg_data)

        # Apply the filters directly on the DataFrame
        ieeg_data = notch_filter(ieeg_data, 59, 61, fs)

        ##############################
        # Calculate synchrony (broadband)
        ##############################
        _, R = calculate_synchrony(ieeg_data.T)
        synchrony_broadband_vector_to_save[interval] = R

        print(f"Finished calculating synchrony for interval {interval}")

    # ##############################
    # # Save the synchrony output
    # ##############################
    # np.save(
    #     os.path.join(SYNCHRONY_broadband_DIRECTORY, f"HUP_{hup_id}.npy"),
    #     synchrony_broadband_vector_to_save,
    # )
    # print(f"Saved synchrony output for HUP {hup_id}")

HUP143_phaseII
Opening HUP143_phaseII with duration 239 hours
Getting iEEG data for interval 717 out of 7890
Finished calculating synchrony for interval 717
Getting iEEG data for interval 718 out of 7890
Finished calculating synchrony for interval 718
Getting iEEG data for interval 719 out of 7890
Finished calculating synchrony for interval 719
Getting iEEG data for interval 720 out of 7890
Finished calculating synchrony for interval 720
Getting iEEG data for interval 721 out of 7890
Finished calculating synchrony for interval 721
Getting iEEG data for interval 722 out of 7890
Finished calculating synchrony for interval 722
Getting iEEG data for interval 723 out of 7890
Finished calculating synchrony for interval 723
Getting iEEG data for interval 724 out of 7890
Finished calculating synchrony for interval 724
Getting iEEG data for interval 725 out of 7890
Finished calculating synchrony for interval 725
Getting iEEG data for interval 726 out of 7890
Finished calculating synchrony for i

In [13]:
synchrony_broadband_vector_to_save

array([nan, nan, nan, ..., nan, nan, nan])

In [14]:
interval

726

In [15]:
# Load SYNCHRONY_broadband_DIRECTORY/HUP_143.npy
test = np.load(os.path.join(SYNCHRONY_broadband_DIRECTORY, "HUP_143.npy"))

In [16]:
test[717:727]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [17]:
synchrony_broadband_vector_to_save[717:727]

array([0.10153952, 0.10376881, 0.10443899, 0.1003387 , 0.10931986,
       0.1040436 , 0.09842504, 0.11012924, 0.10523555, 0.10845476])